In [1]:
#multivariate data preparation

from numpy import array
from numpy import hstack

In [27]:
#split a multivariate sequence into samples

def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        
        #gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [14]:
# define input sequences
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])

In [15]:
in_seq1

array([10, 20, 30, 40, 50, 60, 70, 80, 90])

In [16]:
in_seq2

array([15, 25, 35, 45, 55, 65, 75, 85, 95])

In [17]:
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [18]:
out_seq

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

In [19]:
# convert to [rows, columns] structure

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [20]:
in_seq1

array([[10],
       [20],
       [30],
       [40],
       [50],
       [60],
       [70],
       [80],
       [90]])

In [21]:
in_seq2

array([[15],
       [25],
       [35],
       [45],
       [55],
       [65],
       [75],
       [85],
       [95]])

In [22]:
out_seq

array([[ 25],
       [ 45],
       [ 65],
       [ 85],
       [105],
       [125],
       [145],
       [165],
       [185]])

In [23]:
# horizontally stack columns

dataset = hstack((in_seq1, in_seq2, out_seq))

In [24]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [25]:
# choose a number of time steps

n_steps = 3

In [28]:
# convert into input/ output

X, y = split_sequences(dataset, n_steps)

In [29]:
X

array([[[10, 15],
        [20, 25],
        [30, 35]],

       [[20, 25],
        [30, 35],
        [40, 45]],

       [[30, 35],
        [40, 45],
        [50, 55]],

       [[40, 45],
        [50, 55],
        [60, 65]],

       [[50, 55],
        [60, 65],
        [70, 75]],

       [[60, 65],
        [70, 75],
        [80, 85]],

       [[70, 75],
        [80, 85],
        [90, 95]]])

In [30]:
y

array([ 65,  85, 105, 125, 145, 165, 185])

In [31]:
X.shape

(7, 3, 2)

We can see that the X component has a three-dimensional structure.

The first dimension is the number of samples, in this case 7. The second dimension is the number of time steps per sample, in this case 3, the value specified to the function. Finally, the last dimension specifies the number of parallel time series or the number of variables, in this case 2 for the two parallel series.

In [32]:
y.shape

(7,)

In [33]:
# summarize the data

for i in range(len(X)):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [34]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
 

Using TensorFlow backend.


In [35]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [36]:
n_features

2

In [37]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))

In [38]:
#compile the model

model.compile(optimizer='adam', loss='mse')

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10600     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________


In [40]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [41]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

In [42]:
yhat

array([[206.04333]], dtype=float32)